In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))

In [7]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [8]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],NaN
1,JANUARY,7.0,The Legend of La Llorona,Saban Films,Patricia Harris Seeley (director/screenplay); ...,[3],NaN
2,JANUARY,7.0,The Commando,Saban Films,Asif Akbar (director); Koji Steven Sakai (scre...,[4],NaN
3,JANUARY,7.0,American Siege,Vertical Entertainment,Edward Drake (director/screenplay); Timothy V....,[5],NaN
4,JANUARY,14.0,Scream,Paramount Pictures / Spyglass Media Group,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[6],NaN
...,...,...,...,...,...,...,...
327,DECEMBER,30.0,"Alice, Darling",Lionsgate / Elevation Pictures,Mary Nighy (director); Alanna Francis (screenp...,NaN,[263]
328,DECEMBER,NaN,NaN,NaN,NaN,NaN,NaN
329,DECEMBER,NaN,NaN,NaN,NaN,NaN,NaN
330,DECEMBER,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_2022 = df[['Title']]

In [10]:
!pip install tmdbv3api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from tmdbv3api import TMDb
import json
tmdb = TMDb()
tmdb.api_key = '2d71ac9f8f76e2fe996cdc92cab9f9cd'

In [12]:
from tmdbv3api import Movie
import json
tmdb_movie = Movie() 

In [13]:
def ids(x):
  result = tmdb_movie.search(x)
  if not result:
    return np.NaN
  else:
    return int(result[0].id)

In [14]:
df_2022['id'] = df_2022['Title'].map(lambda x: ids(str(x)))

<ipython-input-14-7a6ad77ada03>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['id'] = df_2022['Title'].map(lambda x: ids(str(x)))


In [15]:
df_2022

,Title,id
0,The 355,522016
1,The Legend of La Llorona,739890
2,The Commando,753232
3,American Siege,766907
4,Scream,646385
...,...,...
327,"Alice, Darling",850028
328,NaN,647808
329,NaN,647808
330,NaN,647808


In [16]:
df_2022[df_2022['id'].isnull()]

,Title,id


In [17]:
def genres(movie_id):
    genres=[]
  
    movie_details = tmdb_movie.details(movie_id)
    movie_genres = movie_details.genres
    if movie_genres:
          genre_str = " " 
          for i in range(0,len(movie_genres)):
              genres.append(movie_genres[i]['name'])
          return genre_str.join(genres)
    else:
          return np.NaN

In [18]:
df_2022['genres'] = df_2022['id'].map(lambda x: genres(int(x)))

<ipython-input-18-7a5021276b22>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['genres'] = df_2022['id'].map(lambda x: genres(int(x)))


In [19]:
df_2022

,Title,id,genres
0,The 355,522016,Action Thriller
1,The Legend of La Llorona,739890,Horror
2,The Commando,753232,Action Crime Thriller
3,American Siege,766907,Action Thriller Crime Drama
4,Scream,646385,Horror Mystery Thriller
...,...,...,...
327,"Alice, Darling",850028,Thriller Drama
328,NaN,647808,Comedy
329,NaN,647808,Comedy
330,NaN,647808,Comedy


In [20]:
def keywords(movie_id):
    keywords=[]

    movie_keywords = tmdb_movie.keywords(movie_id)
    #print(movie_keywords)
    if movie_keywords:
          #print(movie_keywords['keywords'])
          return movie_keywords['keywords']
    else:
          return np.NaN

In [21]:
df_2022['keywords'] = df_2022['id'].map(lambda x: keywords(int(x)))

<ipython-input-21-189d39e975bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['keywords'] = df_2022['id'].map(lambda x: keywords(int(x)))


In [22]:
df_2022

,Title,id,genres,keywords
0,The 355,522016,Action Thriller,"[[id, name], [id, name]]"
1,The Legend of La Llorona,739890,Horror,[]
2,The Commando,753232,Action Crime Thriller,[]
3,American Siege,766907,Action Thriller Crime Drama,[]
4,Scream,646385,Horror Mystery Thriller,"[[id, name], [id, name], [id, name], [id, name..."
...,...,...,...,...
327,"Alice, Darling",850028,Thriller Drama,"[[id, name], [id, name], [id, name], [id, name..."
328,NaN,647808,Comedy,"[[id, name], [id, name], [id, name], [id, name..."
329,NaN,647808,Comedy,"[[id, name], [id, name], [id, name], [id, name..."
330,NaN,647808,Comedy,"[[id, name], [id, name], [id, name], [id, name..."


In [23]:
df_2022['keywords'] = df_2022['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

<ipython-input-23-2e775081c0ee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['keywords'] = df_2022['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [24]:
s = df_2022.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

<ipython-input-24-04a1ff18440f>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = df_2022.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)


based on novel or book    43
duringcreditsstinger      25
woman director            23
aftercreditsstinger       16
based on true story       15
Name: keyword, dtype: int64

In [25]:
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

In [26]:
s = s[s > 1]
stemmer = SnowballStemmer('english')

In [27]:
def keywordsFiltered(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [28]:
df_2022['keywords'] = df_2022['keywords'].apply(keywordsFiltered)
df_2022['keywords'] = df_2022['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

<ipython-input-28-b2f033e30ccf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['keywords'] = df_2022['keywords'].apply(keywordsFiltered)
<ipython-input-28-b2f033e30ccf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['keywords'] = df_2022['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])


In [29]:
df_2022

,Title,id,genres,keywords
0,The 355,522016,Action Thriller,"[spi, agent]"
1,The Legend of La Llorona,739890,Horror,[]
2,The Commando,753232,Action Crime Thriller,[]
3,American Siege,766907,Action Thriller Crime Drama,[]
4,Scream,646385,Horror Mystery Thriller,"[mask, hallucin, murder, serial kil, hospit, s..."
...,...,...,...,...
327,"Alice, Darling",850028,Thriller Drama,"[female friendship, friendship, psychological ..."
328,NaN,647808,Comedy,"[danc, handcuff, jealousi, danc, nightclub, ho..."
329,NaN,647808,Comedy,"[danc, handcuff, jealousi, danc, nightclub, ho..."
330,NaN,647808,Comedy,"[danc, handcuff, jealousi, danc, nightclub, ho..."


In [30]:
df_2022['keywords'] = df_2022['keywords'].map(lambda x: " ".join(x))
df_2022['keywords']

<ipython-input-30-5c83a7189a7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['keywords'] = df_2022['keywords'].map(lambda x: " ".join(x))


0                                              spi agent
1                                                       
2                                                       
3                                                       
4      mask hallucin murder serial kil hospit slasher...
                             ...                        
327    female friendship friendship psychological abu...
328    danc handcuff jealousi danc nightclub hors dub...
329    danc handcuff jealousi danc nightclub hors dub...
330    danc handcuff jealousi danc nightclub hors dub...
331    danc handcuff jealousi danc nightclub hors dub...
Name: keywords, Length: 332, dtype: object

In [31]:
def credits(movie_id):
    cast=[]
  
    movie_credits = tmdb_movie.credits(movie_id)
    if movie_credits['cast'] and movie_credits['crew']:
            for i in movie_credits['crew']:
                if i.get('job') == 'Director':
                    cast.append(i.get('name'))
                    break
            for i in range(0, len(movie_credits['cast'])):
                cast.append(movie_credits['cast'][i]['name'])
            return cast
    else:
            return np.NaN

In [32]:
df_2022['cast'] = df_2022['id'].map(lambda x: credits(int(x)))

<ipython-input-32-d24ecd1803d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['cast'] = df_2022['id'].map(lambda x: credits(int(x)))


In [33]:
df_2022

,Title,id,genres,keywords,cast
0,The 355,522016,Action Thriller,spi agent,"[Simon Kinberg, Jessica Chastain, Lupita Nyong..."
1,The Legend of La Llorona,739890,Horror,,"[David Jones, Elizabeth Selby, Fausto Olmos Re..."
2,The Commando,753232,Action Crime Thriller,,"[Asif Akbar, Michael Jai White, Mickey Rourke,..."
3,American Siege,766907,Action Thriller Crime Drama,,"[Edward Drake, Timothy V. Murphy, Bruce Willis..."
4,Scream,646385,Horror Mystery Thriller,mask hallucin murder serial kil hospit slasher...,"[Matt Bettinelli-Olpin, Melissa Barrera, Mason..."
...,...,...,...,...,...
327,"Alice, Darling",850028,Thriller Drama,female friendship friendship psychological abu...,"[Mary Nighy, Anna Kendrick, Kaniehtiio Horn, C..."
328,NaN,647808,Comedy,danc handcuff jealousi danc nightclub hors dub...,"[Catherine Tate, Mathew Horne, Katherine Parki..."
329,NaN,647808,Comedy,danc handcuff jealousi danc nightclub hors dub...,"[Catherine Tate, Mathew Horne, Katherine Parki..."
330,NaN,647808,Comedy,danc handcuff jealousi danc nightclub hors dub...,"[Catherine Tate, Mathew Horne, Katherine Parki..."


In [34]:
df_2022['cast'] = df_2022['cast'].apply(lambda x: [i for i in x] if isinstance(x, list) else [])
df_2022['cast'] = df_2022['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

<ipython-input-34-e60a8a3611a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['cast'] = df_2022['cast'].apply(lambda x: [i for i in x] if isinstance(x, list) else [])
<ipython-input-34-e60a8a3611a3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['cast'] = df_2022['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])


In [35]:
df_2022

,Title,id,genres,keywords,cast
0,The 355,522016,Action Thriller,spi agent,"[simonkinberg, jessicachastain, lupitanyong'o,..."
1,The Legend of La Llorona,739890,Horror,,"[davidjones, elizabethselby, faustoolmosrenter..."
2,The Commando,753232,Action Crime Thriller,,"[asifakbar, michaeljaiwhite, mickeyrourke, jef..."
3,American Siege,766907,Action Thriller Crime Drama,,"[edwarddrake, timothyv.murphy, brucewillis, ro..."
4,Scream,646385,Horror Mystery Thriller,mask hallucin murder serial kil hospit slasher...,"[mattbettinelli-olpin, melissabarrera, masongo..."
...,...,...,...,...,...
327,"Alice, Darling",850028,Thriller Drama,female friendship friendship psychological abu...,"[marynighy, annakendrick, kaniehtiiohorn, char..."
328,NaN,647808,Comedy,danc handcuff jealousi danc nightclub hors dub...,"[catherinetate, mathewhorne, katherineparkinso..."
329,NaN,647808,Comedy,danc handcuff jealousi danc nightclub hors dub...,"[catherinetate, mathewhorne, katherineparkinso..."
330,NaN,647808,Comedy,danc handcuff jealousi danc nightclub hors dub...,"[catherinetate, mathewhorne, katherineparkinso..."


In [36]:
df_2022['director'] = df_2022['cast'].map(lambda x:x[0] if len(x) >=1 else x)

<ipython-input-36-0af9d3cf81a4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['director'] = df_2022['cast'].map(lambda x:x[0] if len(x) >=1 else x)


In [37]:
df_2022

,Title,id,genres,keywords,cast,director
0,The 355,522016,Action Thriller,spi agent,"[simonkinberg, jessicachastain, lupitanyong'o,...",simonkinberg
1,The Legend of La Llorona,739890,Horror,,"[davidjones, elizabethselby, faustoolmosrenter...",davidjones
2,The Commando,753232,Action Crime Thriller,,"[asifakbar, michaeljaiwhite, mickeyrourke, jef...",asifakbar
3,American Siege,766907,Action Thriller Crime Drama,,"[edwarddrake, timothyv.murphy, brucewillis, ro...",edwarddrake
4,Scream,646385,Horror Mystery Thriller,mask hallucin murder serial kil hospit slasher...,"[mattbettinelli-olpin, melissabarrera, masongo...",mattbettinelli-olpin
...,...,...,...,...,...,...
327,"Alice, Darling",850028,Thriller Drama,female friendship friendship psychological abu...,"[marynighy, annakendrick, kaniehtiiohorn, char...",marynighy
328,NaN,647808,Comedy,danc handcuff jealousi danc nightclub hors dub...,"[catherinetate, mathewhorne, katherineparkinso...",catherinetate
329,NaN,647808,Comedy,danc handcuff jealousi danc nightclub hors dub...,"[catherinetate, mathewhorne, katherineparkinso...",catherinetate
330,NaN,647808,Comedy,danc handcuff jealousi danc nightclub hors dub...,"[catherinetate, mathewhorne, katherineparkinso...",catherinetate


In [38]:
df_2022['actor1']=df_2022['cast'].map(lambda x:x[1] if len(x) >=2 else x)
df_2022['actor2']=df_2022['cast'].map(lambda x:x[2] if len(x) >=3 else x)
df_2022['actor3']=df_2022['cast'].map(lambda x:x[3] if len(x) >=4 else x)

In [39]:
df_2022

,Title,id,genres,keywords,cast,director,actor1,actor2,actor3
0,The 355,522016,Action Thriller,spi agent,"[simonkinberg, jessicachastain, lupitanyong'o,...",simonkinberg,jessicachastain,lupitanyong'o,penélopecruz
1,The Legend of La Llorona,739890,Horror,,"[davidjones, elizabethselby, faustoolmosrenter...",davidjones,elizabethselby,faustoolmosrentería,austinbuchanan
2,The Commando,753232,Action Crime Thriller,,"[asifakbar, michaeljaiwhite, mickeyrourke, jef...",asifakbar,michaeljaiwhite,mickeyrourke,jefffahey
3,American Siege,766907,Action Thriller Crime Drama,,"[edwarddrake, timothyv.murphy, brucewillis, ro...",edwarddrake,timothyv.murphy,brucewillis,robgough
4,Scream,646385,Horror Mystery Thriller,mask hallucin murder serial kil hospit slasher...,"[mattbettinelli-olpin, melissabarrera, masongo...",mattbettinelli-olpin,melissabarrera,masongooding,jennaortega
...,...,...,...,...,...,...,...,...,...
327,"Alice, Darling",850028,Thriller Drama,female friendship friendship psychological abu...,"[marynighy, annakendrick, kaniehtiiohorn, char...",marynighy,annakendrick,kaniehtiiohorn,charliecarrick
328,NaN,647808,Comedy,danc handcuff jealousi danc nightclub hors dub...,"[catherinetate, mathewhorne, katherineparkinso...",catherinetate,mathewhorne,katherineparkinson,nikywardley
329,NaN,647808,Comedy,danc handcuff jealousi danc nightclub hors dub...,"[catherinetate, mathewhorne, katherineparkinso...",catherinetate,mathewhorne,katherineparkinson,nikywardley
330,NaN,647808,Comedy,danc handcuff jealousi danc nightclub hors dub...,"[catherinetate, mathewhorne, katherineparkinso...",catherinetate,mathewhorne,katherineparkinson,nikywardley


In [40]:
df_2022= df_2022.loc[:,['id','director','actor1','actor2','actor3','genres','keywords','Title']]

In [41]:
df_2022 = df_2022.rename(columns={'Title':'title'})

In [42]:
df_2022['title'] = df_2022['title'].str.lower()
df_2022['actor1'] = df_2022['actor1'].str.lower()
df_2022['actor2'] = df_2022['actor2'].str.lower()
df_2022['actor3'] = df_2022['actor3'].str.lower()
# data['director'] = data['director'].str.lower()
df_2022['comb'] = df_2022['genres'] + ' ' + df_2022['keywords'] + ' '  + df_2022['actor1'] + ' '  + df_2022['actor2'] + ' '  + df_2022['actor3'] + ' ' + df_2022['director']

In [43]:
df_2022

,id,director,actor1,actor2,actor3,genres,keywords,title,comb
0,522016,simonkinberg,jessicachastain,lupitanyong'o,penélopecruz,Action Thriller,spi agent,the 355,Action Thriller spi agent jessicachastain lupi...
1,739890,davidjones,elizabethselby,faustoolmosrentería,austinbuchanan,Horror,,the legend of la llorona,Horror elizabethselby faustoolmosrentería aus...
2,753232,asifakbar,michaeljaiwhite,mickeyrourke,jefffahey,Action Crime Thriller,,the commando,Action Crime Thriller michaeljaiwhite mickeyr...
3,766907,edwarddrake,timothyv.murphy,brucewillis,robgough,Action Thriller Crime Drama,,american siege,Action Thriller Crime Drama timothyv.murphy b...
4,646385,mattbettinelli-olpin,melissabarrera,masongooding,jennaortega,Horror Mystery Thriller,mask hallucin murder serial kil hospit slasher...,scream,Horror Mystery Thriller mask hallucin murder s...
...,...,...,...,...,...,...,...,...,...
327,850028,marynighy,annakendrick,kaniehtiiohorn,charliecarrick,Thriller Drama,female friendship friendship psychological abu...,"alice, darling",Thriller Drama female friendship friendship ps...
328,647808,catherinetate,mathewhorne,katherineparkinson,nikywardley,Comedy,danc handcuff jealousi danc nightclub hors dub...,NaN,Comedy danc handcuff jealousi danc nightclub h...
329,647808,catherinetate,mathewhorne,katherineparkinson,nikywardley,Comedy,danc handcuff jealousi danc nightclub hors dub...,NaN,Comedy danc handcuff jealousi danc nightclub h...
330,647808,catherinetate,mathewhorne,katherineparkinson,nikywardley,Comedy,danc handcuff jealousi danc nightclub hors dub...,NaN,Comedy danc handcuff jealousi danc nightclub h...


In [44]:
df_2022.to_csv('movies2022.csv',index=False)

In [45]:
from google.colab import files
files.download("movies2022.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>